IMPORTS ET CHARGEMENTS DES DONNÉES

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import plotly.express as px
import plotly.graph_objects as go

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("../data/processed/online_retail_clean.csv")
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

CONSTRUCTION DES MÉTRIQUES RFM

In [3]:
# Définition de la date de référence pour le calcul de la récence (Recency)
# On ajoute 1 jour à la dernière transaction pour éviter les valeurs nulles
# et garantir que toutes les différences de dates sont positives
reference_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

In [4]:
# Calcul des métriques RFM (Recency, Frequency, Monetary) par client
rfm = (
    df.groupby('Customer_ID')           # Groupement par identifiant client unique
    .agg({                              # Agrégation de plusieurs métriques simultanément
        'InvoiceDate': lambda x: (reference_date - x.max()).days,  # Récence : jours depuis dernier achat
        'InvoiceNo': 'nunique',          # Fréquence : nombre de commandes distinctes
        'Revenue': 'sum'                 # Monétaire : montant total dépensé
    })
    .reset_index()                      # Conversion de l'index en colonne pour obtenir un DataFrame plat
)

# Renommage des colonnes pour correspondre à la terminologie RFM standard
rfm.columns = ['Customer_ID', 'Recency', 'Frequency', 'Monetary']

# Affichage des 5 premiers clients pour vérification
rfm.head()

,Customer_ID,Recency,Frequency,Monetary
0,12346,165,11,372.86
1,12347,3,2,1323.32
2,12348,74,1,222.16
3,12349,43,3,2671.14
4,12351,11,1,300.93


ANALYSE DES DISTRIBUTIONS (AVANT CLUSTERING)

In [5]:
# Création d'un histogramme de distribution de la valeur monétaire des clients (M)
fig = px.histogram(
    rfm,                        # DataFrame RFM contenant les scores clients
    x='Monetary',               # Variable à distribuer : montant total dépensé par client
    nbins=50,                   # Nombre d'intervalles pour la distribution
    title='Monetary Distribution'  # Titre du graphique
)

# Application du thème visuel épuré avec fond blanc
fig.update_layout(template="plotly_white")

fig.write_image("../reports/figures/Monetary_Distribution.png")
# Affichage interactif du graphique
fig.show()

NORMALISATION DES VARIABLES

In [6]:
# Préparation des données pour le clustering K-Means
# Standardisation obligatoire car K-Means est sensible aux échelles différentes
scaler = StandardScaler()           # Initialisation du scaler (moyenne=0, écart-type=1)
rfm_scaled = scaler.fit_transform(  # Ajustement et transformation des données
    rfm[['Recency', 'Frequency', 'Monetary']]  # Sélection des 3 dimensions RFM
)

CHOIX DU NOMBRE DE CLUSTERS (ELBOW METHOD)

In [7]:
inertia = []                        # Liste pour stocker les valeurs d'inertie
for k in range(2, 11):              # Test de 2 à 10 clusters
    kmeans = KMeans(
n_clusters=k,               # Nombre de clusters à tester
random_state=42             # Reproductibilité des résultats
)
kmeans.fit(rfm_scaled)          # Entraînement du modèle sur données standardisées
inertia.append(kmeans.inertia_) # Stockage de l'inertie (somme des distances au centroïde)

In [8]:
# Visualisation de la méthode du coude pour déterminer le nombre optimal de clusters
fig = px.line(
    x=list(range(2, len(inertia) + 2)),  # Axe X : nombre de clusters testés
    y=inertia,                           # Axe Y : inertie
    markers=True,                        # Affiche un point pour chaque valeur
    title='Elbow Method for Optimal K'   # Titre du graphique
)

# Personnalisation des axes et du thème
fig.update_layout(
    xaxis_title="Number of clusters",   # Libellé axe X
    yaxis_title="Inertia",              # Libellé axe Y
    template="plotly_white"             # Thème visuel épuré
)

# Affichage interactif du graphique
fig.show()

CLUSTERING FINAL

In [9]:
# Entraînement du modèle K-Means avec 4 clusters et attribution des segments
kmeans = KMeans(
n_clusters=4,           # Nombre optimal de clusters déterminé par la méthode du coude
random_state=42         # Reproductibilité des résultats
)
# Prédiction des clusters et ajout au DataFrame RFM
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)  # fit_predict : entraîne et assigne chaque client à un cluster

INTERPRÉTATIONS DES CLUSTERS

In [10]:
# Analyse descriptive des clusters : calcul des moyennes RFM et effectifs
cluster_summary = (
rfm.groupby('Cluster')              # Groupement par numéro de cluster attribué
.agg({                              # Agrégation multivariée par cluster
'Recency': 'mean',              # Récence moyenne : clients récents ou éloignés
'Frequency': 'mean',            # Fréquence moyenne : clients habitués ou occasionnels
'Monetary': 'mean',             # Valeur monétaire moyenne : gros ou petits dépensiers
'Customer_ID': 'count'          # Taille du cluster : nombre de clients dans le segment
})
.reset_index()                      # Réinitialisation de l'index pour un DataFrame plat
)
# Affichage du tableau récapitulatif des segments
cluster_summary

,Cluster,Recency,Frequency,Monetary,Customer_ID
0,0,43.031835,4.455056,1710.650030,3204
1,1,242.976122,1.659981,593.540319,1047
2,2,5.600000,113.600000,215535.000000,5
3,3,14.910714,47.017857,28896.416661,56


VISUALISATIONS DES SEGMENTS

In [11]:
# Visualisation des segments clients en nuage de points interactif (RFM)
fig = px.scatter(
rfm,                        # DataFrame contenant les scores RFM et clusters
x='Recency',                # Axe X : récence (jours depuis dernier achat)
y='Monetary',               # Axe Y : valeur monétaire (montant total dépensé)
color='Cluster',            # Couleur des points selon le cluster attribué
size='Frequency',           # Taille des points proportionnelle à la fréquence d'achat
title='Customer Segments (RFM Clustering)'  # Titre du graphique
)
# Application du thème visuel épuré
fig.update_layout(template="plotly_white")

fig.write_image("../reports/figures/Customer_Segments.png")
# Affichage interactif du graphique
fig.show()

TRADUCTION BUSINESS

## Interprétation des segments de clientèle

- Groupe 0 : Clients fidèles à forte valeur ajoutée → fidélisation et offres VIP

- Groupe 1 : Clients récents à faible budget → opportunités de vente additionnelle

- Groupe 2 : Clients inactifs → campagnes de réactivation

- Groupe 3 : Clients réguliers à budget modéré → ventes croisées

EXPORT POUR LA SUITE (ML & APP)

In [12]:
rfm.to_csv("../data/processed/rfm_segmentation.csv", index=False)